In [15]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/my_custom_model_name.h5'
model = load_model(model_path)

print("✅ Model loaded successfully from Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


✅ Model loaded successfully from Drive


In [17]:
from google.colab import files
from tensorflow.keras.preprocessing import image
from IPython.display import clear_output
import ipywidgets as widgets
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os, zipfile
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image as PILImage

img_size = (224, 224)
# Example mapping; replace with your actual class labels and order
index_to_class = {0: 'Electricity', 1: 'Roads', 2: 'Sanitation', 3: 'Water'}

# ✅ Step 9: Initialize complaint database
complaints = {
    'Roads': [],
    'Water': [],
    'Electricity': [],
    'Sanitation': []
}

# ✅ Step 10: Dashboard
def show_dashboard():
    clear_output()
    print("📊 Department-wise Complaint Dashboard:\n")
    for dept in complaints:
        dept_complaints = complaints[dept]
        print(f"🏢 {dept}: {len(dept_complaints)} complaint(s)")
        if not dept_complaints:
            print("  ❌ No complaints yet.")
        else:
            for i, c in enumerate(dept_complaints, 1):
                print(f"  {i}. {c['Name']} | {c['Phone']} | {c['Location']} | {c['Description']}")
        print("-" * 50)

# ✅ Step 11: View All Complaints
def view_all_complaints():
    clear_output()
    print("📋 All Registered Complaints:\n")
    count = 0
    for dept, comp_list in complaints.items():
        for comp in comp_list:
            count += 1
            print(f"{count}. [{dept}] {comp['Name']} - {comp['Phone']} - {comp['Location']} - {comp['Description']}")
    if count == 0:
        print("❌ No complaints registered.")
    print("-" * 50)

# ✅ Step 12: Complaint Registration Menu
def complaint_menu():
    clear_output()
    print("📝 Fill Complaint Details Below:")

    name = widgets.Text(description="Name:")
    phone = widgets.Text(description="Phone:")
    location = widgets.Text(description="Location:")
    description = widgets.Textarea(description="Issue:")
    upload_btn = widgets.Button(description="Upload Image", button_style="success")

    display(name, phone, location, description, upload_btn)

    def on_upload(b):
        clear_output()
        print("📸 Upload an image of the complaint (jpg/png)")
        uploaded_img = files.upload()

        for img_name in uploaded_img:
            try:
                img = image.load_img(img_name, target_size=img_size)
                img_array = image.img_to_array(img) / 255.0
                img_array = np.expand_dims(img_array, axis=0)

                pred = model.predict(img_array)
                class_index = np.argmax(pred)
                predicted_dept = index_to_class[class_index]

                print(f"\n🏢 Predicted Department: {predicted_dept}")
                display(PILImage.open(img_name))

                # Confirm complaint registration
                confirm = widgets.RadioButtons(options=["Yes", "No"], description="Register?")
                confirm_btn = widgets.Button(description="Submit", button_style="primary")

                def on_confirm(c):
                    clear_output()
                    if confirm.value == "Yes":
                        data = {
                            "Name": name.value,
                            "Phone": phone.value,
                            "Location": location.value,
                            "Description": description.value
                        }
                        complaints[predicted_dept].append(data)
                        print("✅ Complaint Registered!\n")
                    else:
                        print("❌ Complaint Not Registered.\n")
                    show_options_menu()

                display(confirm, confirm_btn)
                confirm_btn.on_click(on_confirm)

            except Exception as e:
                print("❌ Error during prediction:", e)

    upload_btn.on_click(on_upload)

# ✅ Step 13: Main Options Menu
def show_options_menu():
    print("📋 Choose an option:")
    option = widgets.RadioButtons(
        options=[
            "Register Another Complaint",
            "View All Complaints",
            "Dashboard",
            "Exit"
        ],
        description="Options:"
    )
    next_btn = widgets.Button(description="Proceed", button_style="info")

    def on_next(b):
        clear_output()
        if option.value == "Register Another Complaint":
            complaint_menu()
        elif option.value == "View All Complaints":
            view_all_complaints()
            show_options_menu()
        elif option.value == "Dashboard":
            show_dashboard()
            show_options_menu()
        elif option.value == "Exit":
            print("👋 Exiting. Thank you!")
        else:
            print("❌ Invalid option.")

    display(option, next_btn)
    next_btn.on_click(on_next)

# ✅ Start the application
complaint_menu()

📋 All Registered Complaints:

1. [Electricity] sai - 4566 - V R colony - Dustin
--------------------------------------------------
📋 Choose an option:


RadioButtons(description='Options:', options=('Register Another Complaint', 'View All Complaints', 'Dashboard'…

Button(button_style='info', description='Proceed', style=ButtonStyle())